<div id="header_wrap" class="outer">
    <header class="inner">
      <h1 id="project_title">Checking Finite Algebra Benchmark Solutions for the IGLTI and Modified ILTI Algorithms</h1>
    </header>
</div>

<div style="padding-left:1em; padding-top:1em; padding-right:1em; padding-bottom:1em; border-style: solid; border-width: 5px; border-color: #212121;">

<div>
    <div style="float:left;display:inline-block">
        <a href="http://robynffrancon.com" target="_blank">Robyn Ffrancon</a>
    </div>
    <div style="float:right; ">
        <div style="">rffrancon@gmail.com</div>
    </div>
</div>

<h2 id='abstract' style="padding-bottom:1em; padding-top:1em">Abstract</h2>
In this notebook the solutions obtained when testing the Iterative Greedy Local Tree Improvement (<b>IGLTI</b>) and modified Iterative Local Tree Improvement (<b>ILTI</b>) algorithms on standard finite algebra benchmarks will be checked. These results were documented in <a href='#ffrancon_and_schoenauer_iglti'>[1]</a>.

The benchmarks presented in this notebook have been studied and clearly defined in <a href='#lee_etal_2008'>[2]</a> and <a href='#krzysztof_and_oreilly_2014'>[3]</a>.
</div>

<h2 id='contents'>Contents</h2>
1. <a href='#background'>Background</a>
2. <a href='#defining-benchmarks'>Defining Benchmarks</a>
3. <a href='#loading-data'>Loading Data</a>
4. <a href='#checking-solutions'>Checking Solutions</a>
5. <a href='#references'>References</a>

<hr style="height:1px;border:none;color:#333;background-color:#333;">

<div style='display:inline-block;'><font>[^](#contents)</font></div><h2 style='display:inline-block;' id='background'>Background</h2>

Solutions found using the IGLTI and modified ILTI algorithms were saved as their string representations, for example:
<i>a1(a1(ARG2, ARG1), ARG2)</i>.

They will be executed using Python's built-in <a href='https://docs.python.org/2/library/functions.html#eval' target="_blank"><i>eval</i></a> function. The operator(s) (<i>a1</i> in the example above) and input variables (<i>ARG1</i> and <i>ARG2</i> in the example above) used within each solution will be defined within the global scope of the Python code. The values of the input variables will be changed for each fitness case.

<div style='display:inline-block;'><font>[^](#contents)</font></div><h2 style='display:inline-block;' id='defining-benchmarks'>Defining Benchmarks</h2>

In the first part of this section, the <i>discriminator</i> term and <i>Mal'cev</i> term objective function are defined using Python:

In [1]:
def objective_func_discrim(input_pattern):
    """This is the objective function for the catagorical
    discriminator term.

    :param input_pattern: A single fitness test with three values.
    :returns: The target output for the fitness test.
    """

    x, y, z = input_pattern

    if x != y:
        return x
    elif x == y:
        return z

    print('ERROR objective_func_discrim')
    exit()

def objective_func_malcev(input_pattern):
    """This is the objective function for the catagorical
    Mal'cev term.

    :param input_pattern: A single fitness test with three values.
    :returns: The target output for the fitness test.
    """

    if input_pattern[0] == input_pattern[1]:
        return input_pattern[2]

    elif input_pattern[1] == input_pattern[2]:
        return input_pattern[0]

    print('ERROR objective_func_malcev')
    exit()

Next, the finite algebra operators that were examined are defined as simple Python functions. The original definitions for these matrices were obtained from <a href='#lee_etal_2008'>[2]</a>.

In [2]:
def a1(b, c):
    return [
        [2, 1, 2],
        [1, 0, 0],
        [0, 0, 1]
    ][b][c]

def a2(b, c):
    return [
        [2, 0, 2],
        [1, 0, 2],
        [1, 2, 1]
    ][b][c]

def a3(b, c):
    return [
        [1, 0, 1],
        [1, 2, 0],
        [0, 0, 0]
    ][b][c]

def a4(b, c):
    return [
        [1, 0, 1],
        [0, 2, 0],
        [0, 1, 0]
    ][b][c]

def a5(b, c):
    return [
        [1, 0, 2],
        [1, 2, 0],
        [0, 1, 0]
    ][b][c]

<div style='display:inline-block;'><font>[^](#contents)</font></div><h2 style='display:inline-block;' id='loading-data'>Loading Data</h2>

At this point, it is necassary to define a function for loading the solutions which have been previously saved. There are three different sets of solutions, one for each scheme that was tested: ILTI, IGLTI (depth 3), and IGLTI (depth 2). Each of these three schemes are detailed in <a href='#ffrancon_and_schoenauer_iglti'>[1]</a>. The solutions obtained for each scheme are stored in three separate directories.

Given a scheme directory and a benchmark name, the function <i>get_solutions</i> loads the corresponding <a href='https://docs.python.org/2/library/pickle.html' target="_blank">pickled</a> solutions file. It also extracts and returns the string representations of the solution programs.

In [3]:
import itertools

from os import listdir
from os.path import isfile, join

import pickle

def get_solutions(benchmark_name, scheme_dir):
    # get all file names in base_path directory
    fnames = [f for f in listdir(scheme_dir) if isfile(join(scheme_dir, f))]
    
    # find the right file given the benchmark
    found_fname = None
    for fname in fnames:
        if fname.find(benchmark_name) != -1:
            found_fname = fname
            break
    
    # load solutions data file
    solutions = pickle.load(open(scheme_dir + found_fname, 'rb'))
    
    # collect final solution program trees as strings
    solution_programs = [solution['tree_str'] for solution in solutions]
    return solution_programs

<div style='display:inline-block;'><font>[^](#contents)</font></div><h2 style='display:inline-block;' id='checking-solutions'>Checking Solutions</h2>

The first function in this section calculates all of the input patterns for a given benchmark:

In [4]:
def gen_input_patterns(benchmark_name):
    if benchmark_name in ['D_A1', 'D_A2', 'D_A3', 'D_A4', 'D_A5']:
        input_patterns = [list(seq) for seq in itertools.product([0, 1, 2], repeat=3)]
    
    elif benchmark_name in ['M_A1', 'M_A2', 'M_A3', 'M_A4', 'M_A5']:
            temp = [list(seq) for seq in itertools.product([0, 1], repeat=3)]
            temp += [list(seq) for seq in itertools.product([0, 2], repeat=3)]
            temp += [list(seq) for seq in itertools.product([1, 2], repeat=3)]

            input_patterns = []
            for inp in temp:
                if (inp[0] == inp[2]) and (inp[1] != inp[0]):
                    continue
                if inp in input_patterns:
                    continue
                input_patterns.append(inp)
                
    return input_patterns

Given a potential solution and input patterns, <i>eval_solution</i> calculates its output values.

In [5]:
def eval_solution(solution, input_pattern):
    # define arguments
    for i_bit, bit_val in enumerate(input_pattern):
        globals()['ARG' + str(i_bit)] = bit_val
    
    # evaluate solution
    return eval(solution)

The function <i>test_solutions</i> examines the validity of a set of solutions. It returns a list of True or False values which indicate the validity of each solution.

In [6]:
def test_solutions(solutions, input_patterns, target_outputs):
    
    results = []
    for solution in solutions:
        solution_outputs = [eval_solution(solution, input_pattern) 
                            for input_pattern in input_patterns]

        for solution_output, target_output in zip(solution_outputs, target_outputs):
            if solution_output != target_output:
                results.append(False)
                break
                
        results.append(True)

    return results

Given a scheme directory, the <i>check_scheme_results</i> function, examines the validity of each solution for each benchmark.

In [7]:
def check_scheme_results(scheme_dir):
    benchmark_names = ['D_A1', 'D_A2', 'D_A3', 'D_A4', 'D_A5', 
                       'M_A1', 'M_A2', 'M_A3', 'M_A4', 'M_A5']

    all_results = []
    for benchmark_name in benchmark_names:
        # get solutions
        solutions = get_solutions(benchmark_name, scheme_dir)
    
        # generate input patterns
        input_patterns = gen_input_patterns(benchmark_name)
    
        # chose objective function
        if benchmark_name in ['D_A1', 'D_A2', 'D_A3', 'D_A4', 'D_A5']:
            objective_func = objective_func_discrim
        elif benchmark_name in ['M_A1', 'M_A2', 'M_A3', 'M_A4', 'M_A5']:
            objective_func = objective_func_malcev
        
        # get target outputs
        target_outputs = [objective_func(input_pattern)
            for input_pattern in input_patterns]
    
        # test solutions
        benchmark_results = test_solutions(solutions, input_patterns, target_outputs)
        all_results += benchmark_results
        
    return all_results

The remaining code makes use of all of the above code to evaluate the validity of each solution obtained using each scheme.

In [8]:
scheme_names = ['finite_algebra_solutions_ILTI', 
                'finite_algebra_solutions_IGLTI_depth3', 
                'finite_algebra_solutions_IGLTI_depth2']

for scheme_name in scheme_names:
    all_results = check_scheme_results(scheme_name + '/')
    
    print('Scheme: ', scheme_name)
    print('Number of results:', len(all_results))
    
    if False in all_results:
        print('one or more False solutions')
    else:
        print('all results correct')
    
    print('')

Scheme:  finite_algebra_solutions_ILTI
Number of results: 200
all results correct

Scheme:  finite_algebra_solutions_IGLTI_depth3
Number of results: 200
all results correct

Scheme:  finite_algebra_solutions_IGLTI_depth2
Number of results: 200
all results correct



<hr style="height:1px;border:none;color:#333;background-color:#333;">
<div style='display:inline-block;'><font>[^](#contents)</font></div><h2 style='display:inline-block;' id='acknowledgements'>Acknowledgements</h2>

I wish to thank <a href="https://www.lri.fr/~marc/" target="_blank">Marc Schoenauer</a> for interesting discussions and excellent supervision during my 6 month masters internship at INRIA, Paris, France where this work was conducted.

<div style='display:inline-block;'><font>[^](#contents)</font></div><h3 style='display:inline-block;' id='references'>References</h3>

<a href='http://robynffrancon.com/Greedy%20Semantic%20Local%20Search%20for%20Small%20Solutions.pdf' target="_blank" id='ffrancon_and_schoenauer_iglti'>1.</a> Ffrancon, Robyn, and Marc Schoenauer. "Greedy Semantic Local Search for Small Solutions.", Semantic Methods in Genetic Programming Workshop, GECCO 2015.

<a href='http://dl.acm.org/citation.cfm?id=1389343' target="_blank" id='lee_etal_2008'>2.</a> Spector, Lee, et al. "Genetic programming for finite algebras." Proceedings of the 10th annual conference on Genetic and evolutionary computation. ACM, 2008.

<a href='http://dl.acm.org/citation.cfm?id=2598288' target="_blank" id='krzysztof_and_oreilly_2014'>3.</a> Krawiec, Krzysztof, and Una-May O'Reilly. "Behavioral programming: a broader and more detailed take on semantic GP." Proceedings of the 2014 conference on Genetic and evolutionary computation. ACM, 2014.

In [9]:
# sets the Notebook style
# css style sheet obtained from: https://github.com/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers
from IPython.core.display import HTML

def css_styling():
    styles = open("styles/custom.css", "r").read()
    return HTML(styles)
css_styling()